In [14]:
#!pip install agentpy owlready2

In [15]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
from owlready2 import *

In [16]:
onto = get_ontology("file://onto.owl")

In [17]:
#ONTOLOGIA
#onto.delete()
with onto:

    class Entity(Thing):
        pass

    class Drone(Entity):
        pass

    class Place(Thing):
        pass

    class Position(Thing):
        pass

    class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]

    class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]

        pass

In [ ]:
class DroneAgent(ap.Agent):

    #Razonamiento
    def see(self, e):
      self.per = []
      own_position = str(e.positions[self])

      # Creación de percepto: lista de creencias del dron
      self.per.append(Drone(has_place=Place(has_position=str(own_position))))

      return self.per

    def brf():
      pass

    def opt():
      pass

    def filter():
      pass

    def plan():
      pass

    def next(self, per):
      pass

    def action(self, act):
      pass

    #Simulación de Agente
    def setup(self):
        """
        Inicialización del agente
        """
        self.agentType = 0 # Tipo de agente
        self.directionTag = 'N' # Dirección inicial
        self.direction = (-1,0) # Dirección inicial

        # Acciones del agente
        self.actions = (
            self.move,

        )

        # Reglas del agente
        self.rules = (

        )

        #Deseos del agente
        self.desires= (
          ((10,3),0)
        )

        #First step
        self.first_step = True

    def step(self):
        """
        Función paso a paso
        """
        self.see(self.model.grid) #Percepción del Grid
        self.next() #Razonamiento y acción

"""
        if self.first_step:
            self.idle()
            self.first_step = False
        else:
            self.e = self.get_env(self.model.grid)
            self.per = self.see(self.e)

            self.act = self.next(self.per)
            if self.act is not None:
                self.action(self.act)
            print(f"Direction: {self.direction}")
            print(f"Action: {self.act.__name__ if self.act else 'None'}")
"""
        pass

    def update(self):
        pass

    def end(self):
        pass

    #INTENTIONS
    #def filter(self, desires):



    #ACTIONS

    #For environment perception

    """
    def get_env(self, env):
        neighbors = env.neighbors(self, 1)
        current_position = env.positions[self]
        self.e = self.transform_neighbors(neighbors, current_position)
        if not self.e:
            self.e = [('N', '0'), ('S', '0'), ('W', '0'), ('E', '0')]
        return self.e

    def transform_neighbors(self, neighbors, current_position):
        directions = ['N', 'S', 'W', 'E']
        result = {d: '0' for d in directions}  # Initialize all directions with '0'

        for direction in directions:
            if direction == 'N':
                position = (current_position[0] - 1, current_position[1])
            elif direction == 'S':
                position = (current_position[0] + 1, current_position[1])
            elif direction == 'W':
                position = (current_position[0], current_position[1] - 1)
            elif direction == 'E':
                position = (current_position[0], current_position[1] + 1)

            if self.is_wall(position):
                result[direction] = "Wall"
            else:
                cell_content = self.model.grid.agents[position]
                if cell_content:
                    # Use next() to get the first agent if there is one
                    agent = next(iter(cell_content), None)
                    if agent and hasattr(agent, 'agentType'):
                        if str(agent.agentType) == "0":
                            result[direction] = "Bot"
                        elif str(agent.agentType) == "1":
                            result[direction] = "Object"
                        elif str(agent.agentType) == "2":
                            result[direction] = "Shelf"
                        else:
                            result[direction] = "0"
                    else:
                        result[direction] = "0"
                else:
                    result[direction] = "0"  # Empty cell

        return list(result.items())

    def is_wall(self, position):
        x, y = position
        return x < 0 or x >= self.model.p.M or y < 0 or y >= self.model.p.N
"""


    #For planning
    def findPath(self):
        return tuple(a - b for a, b in zip( (10,3), self.model.grid.positions[self] ))



    #For affecting the environment
    def move(self):
        if self.directionTag == 'N':
            self.model.grid.move_by(self, (-1, 0))
        if self.directionTag == 'S':
            self.model.grid.move_by(self, (1, 0))
        if self.directionTag == 'E':
            self.model.grid.move_by(self, (0, 1))
        if self.directionTag == 'W':
            self.model.grid.move_by(self, (0, -1))
        print(f"Moved to new position: {self.model.grid.positions[self]}")
        pass

    def turn(self):
        if self.directionTag == "N":
            self.direction = (0,1) #Hacia Este
            self.directionTag = "E"
        elif self.directionTag == "E":
            self.direction = (1,0)  #Hacia Sur
            self.directionTag = "S"
        elif self.directionTag == "S":
            self.direction = (0,-1) #Hacia Oeste
            self.directionTag = "W"
        elif self.directionTag == "W":
            self.direction = (-1,0) #Hacia Norte
            self.directionTag = "N"
        pass

    def idle(self):
        pass

In [18]:
class BuildingAgent(ap.Agent):

    def setup(self):
        self.agentType = 2
        pass

    def step(self):
        pass

    def update(self):
        pass

    def end(self):
        pass

In [19]:
class PrisionModel(ap.Model):
    def setup(self):
        self.drones = ap.AgentList(self, self.p.drones, DroneAgent)
        self.buildings = ap.AgentList(self, self.p.buildings, BuildingAgent)

        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.grid.add_agents(self.drones, positions=(self.p.dpos), random=False, empty=True)
        self.grid.add_agents(self.buildings, positions=(self.p.bpos), random=False, empty=True)
        pass

    def step(self):
        self.drones.step()
        pass

    def update(self):
        pass

    def end(self):
        pass

In [20]:
parameters = {
    'M': 14,
    'N': 20,
    'steps': 20,
    'drones': 1,
    'dpos': ((10,7),),
    #'boxes': 30,
    'buildings': 49,
    'bpos': ((2,4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13),(2, 14), (2, 15), (2, 16),
             (3,4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13),(3, 14), (3, 15), (3, 16),
             (4,4), (4,5), (5,4), (5,5), (6,4), (6,5), (7,4), (7,5),
             (4,15), (4,16), (5,15), (5,16), (6,15), (6,16), (7,15), (7,16),
             (11, 7), (11, 8), (11, 9), (11, 10), (11, 11), (11, 12), (11, 13))
}

# model = WarehouseModel(parameters)
# results = model.run()

In [21]:
#ANIMATION
def animation_plot(model, ax):
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Warehouse \n Time-step: {model.t}, "
                 f"Boxes: {0}")


In [22]:
#SIMULATION

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = PrisionModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())